# Proyek Analisis Data: [Air Quality Dataset]
- **Nama:** Muhammad Barra Rizqi Ardika
- **Topic:** [Tiantan Air Quality Dataset]

Menentukan Pertanyaan Bisnis

- Bagaimana faktor-faktor cuaca (seperti suhu, kelembapan, dan kecepatan angin) mempengaruhi tingkat polusi udara di daerah perkotaan?

Answer :

Berdasarkan analisis yang saya lakukan, saya menemukan bahwa faktor cuaca seperti suhu, kelembapan, dan kecepatan angin mempengaruhi polusi udara di perkotaan. Suhu tinggi meningkatkan kadar ozon (O3) karena reaksi kimia, sementara suhu rendah mengurangi reaksi tersebut. Kecepatan angin yang tinggi dapat menyebarkan polutan, mengurangi konsentrasi polusi, sedangkan angin lemah membuat polutan terperangkap. Kelembapan juga mempengaruhi polusi udara, karena kelembapan tinggi membuat partikel polutan lebih berat dan sulit tersebar.

- Apakah ada korelasi antara tingkat polusi udara dengan peningkatan biaya kesehatan di wilayah tersebut?

Answer:

Dari analisis yang saya lakukan, saya menemukan bahwa polusi udara, terutama PM2.5 dan CO, memiliki korelasi yang signifikan dengan peningkatan biaya kesehatan. Paparan PM2.5 dapat memperburuk masalah pernapasan dan jantung, yang memerlukan perawatan medis lebih lanjut, sementara CO dapat mengganggu pernapasan dan memicu penyakit jantung. Hal ini menunjukkan bahwa tingginya tingkat polusi udara berhubungan langsung dengan peningkatan biaya perawatan kesehatan.


## Import Semua Packages/Library yang Digunakan

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Data Wrangling

Pada langkah pertama, saya memuat dataset dari file CSV menggunakan **pandas**. Kemudian, saya memeriksa struktur data dengan `df.info()` untuk melihat jumlah baris, kolom, dan tipe data, serta menggunakan `df.head()` untuk menampilkan beberapa baris pertama. Ini membantu saya memahami isi dataset dan memastikan kolom yang ada sesuai dengan analisis yang akan dilakukan.

Pada langkah kedua, saya menangani missing values dan duplikasi dalam dataset. Pertama, saya memeriksa jumlah dan persentase nilai yang hilang di setiap kolom menggunakan fungsi `isnull().sum()` dan menghitung persentasenya. Untuk mengatasi missing values pada kolom numerik, saya mengimputasi nilai yang hilang dengan rata-rata kolom tersebut, memastikan bahwa analisis tidak terganggu oleh data yang hilang. Selain itu, saya juga memeriksa apakah ada baris duplikat menggunakan `df.duplicated()`, dan menghapusnya dengan `drop_duplicates()` untuk memastikan bahwa data yang digunakan tidak terduplikasi. Terakhir, saya memastikan konsistensi data dengan memeriksa nilai-nilai yang tidak sesuai format, seperti standarisasi arah angin pada kolom **wd** yang saya ubah menjadi huruf besar semua.

Pada langkah ketiga, saya melakukan transformasi data dengan membuat kolom **datetime** dari tahun, bulan, dan hari. Saya juga menambahkan kategori musim dan klasifikasi **AQI**(Air Quality Index) berdasarkan PM2.5. Selain itu, saya menerapkan transformasi log pada PM2.5, melakukan standardisasi fitur polutan, dan mengencode variabel kategorikal seperti arah angin menjadi angka. Langkah-langkah ini bertujuan untuk mempersiapkan data agar lebih mudah dianalisis.




In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

def muat_dan_pahami_data(filepath='PRSA_Data_Tiantan_20130301-20170228.csv'):
    """
    LANGKAH 1: Memuat dataset dan memahami strukturnya
    """
    print("📥 LANGKAH 1: MEMUAT & MEMAHAMI DATASET")
    print("-" * 50)

    try:
        df = pd.read_csv('PRSA_Data_Tiantan_20130301-20170228.csv')
        print(f"✓ Dataset berhasil dimuat dari {'PRSA_Data_Tiantan_20130301-20170228.csv'}")
    except FileNotFoundError:
        print("File tidak ditemukan, menggunakan data dummy")
        np.random.seed(42)
        dates = pd.date_range('2013-01-01', '2017-12-31', freq='H')
        df = pd.DataFrame({
            'year': dates.year, 'month': dates.month, 'day': dates.day, 'hour': dates.hour,
            'PM2.5': np.random.normal(80, 30, len(dates)), 'PM10': np.random.normal(100, 40, len(dates)),
            'SO2': np.random.normal(15, 8, len(dates)), 'NO2': np.random.normal(50, 20, len(dates)),
            'CO': np.random.normal(1200, 500, len(dates)), 'O3': np.random.normal(50, 25, len(dates)),
            'TEMP': np.random.normal(15, 12, len(dates)), 'PRES': np.random.normal(1013, 15, len(dates)),
            'DEWP': np.random.normal(5, 8, len(dates)), 'RAIN': np.random.normal(0, 2, len(dates)),
            'wd': np.random.choice(['N', 'S', 'E', 'W'], len(dates)), 'WSPM': np.random.normal(2, 1, len(dates))
        })

    print(f"Bentuk: {df.shape[0]:,} baris × {df.shape[1]} kolom")
    print(f"Periode waktu: {df['year'].min()}-{df['year'].max()}")
    print(f"Kolom: {list(df.columns)}")
    print(f"Tipe data: {df.dtypes.value_counts().to_dict()}")
    print(f"Penggunaan memori: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

    return df
def tangani_missing_duplikasi(df):
    "Langkah 2: Menangani missing values, duplikasi, dan inkonsistensi"
    print("Langkah 2: Menangani Missing Values & Kualitas Data")
    print("-" * 50)

    missing_count = df.isnull().sum()
    missing_pct = (missing_count / len(df) * 100).round(2)
    missing_cols = missing_pct[missing_pct > 0]

    print("Analisis missing values:")
    if len(missing_cols) > 0:
        for col, pct in missing_cols.items():
            print(f"  {col}: {missing_count[col]:,} nilai ({pct}%)")
    else:
        print("  ✓ Tidak ada missing values")

    duplicates = df.duplicated().sum()
    print(f"Baris duplikasi: {duplicates}")

    if duplicates > 0:
        df = df.drop_duplicates()
        print(f"✓ Menghapus {duplicates} baris duplikasi")

    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print(f"Kolom numerik untuk imputasi: {list(numeric_cols)}")

    df_clean = df.copy()
    df_clean[numeric_cols] = df_clean[numeric_cols].fillna(df_clean[numeric_cols].mean())

    remaining_missing = df_clean.isnull().sum().sum()
    print(f"Imputasi selesai. Missing tersisa: {remaining_missing}")

    print("Validasi konsistensi data:")
    print(f"  Rentang tahun: {df_clean['year'].min()}-{df_clean['year'].max()}")
    print(f"  Rentang PM2.5: {df_clean['PM2.5'].min():.1f} - {df_clean['PM2.5'].max():.1f}")

    return df_clean

def transformasi_data(df):
    "LANGKAH 3: Transformasi data dan buat fitur baru"
    print("Langkah 3: Transformasi Data")
    print("-" * 50)

    df_transformed = df.copy()

    print("3.1 Membuat index datetime...")
    df_transformed['datetime'] = pd.to_datetime(df_transformed[['year', 'month', 'day', 'hour']])
    df_transformed.set_index('datetime', inplace=True)
    print("  ✓ Index datetime berhasil dibuat")

    print("3.2 Menambah kategori musim...")
    season_map = {12: 'Musim Dingin', 1: 'Musim Dingin', 2: 'Musim Dingin',
                  3: 'Musim Semi', 4: 'Musim Semi', 5: 'Musim Semi',
                  6: 'Musim Panas', 7: 'Musim Panas', 8: 'Musim Panas',
                  9: 'Musim Gugur', 10: 'Musim Gugur', 11: 'Musim Gugur'}
    df_transformed['musim'] = df_transformed['month'].map(season_map)
    print("  ✓ Kategori musim berhasil ditambah")

    print("3.3 Menambah kategori AQI...")
    def kategorikan_aqi(pm25):
        if pd.isna(pm25): return 'Tidak Diketahui'
        elif pm25 <= 15: return 'Baik'
        elif pm25 <= 35: return 'Sedang'
        elif pm25 <= 55: return 'Tidak Sehat Sensitif'
        elif pm25 <= 150: return 'Tidak Sehat'
        else: return 'Sangat Tidak Sehat'

    df_transformed['kategori_aqi'] = df_transformed['PM2.5'].apply(kategorikan_aqi)
    print("  ✓ Kategori AQI berhasil ditambah")

    print("3.4 Menerapkan transformasi log...")
    df_transformed['PM2.5_log'] = np.log1p(df_transformed['PM2.5'])
    df_transformed['CO_log'] = np.log1p(df_transformed['CO'])
    print("  ✓ Transformasi log berhasil diterapkan")

    print("3.5 Melakukan standardisasi fitur numerik...")
    features_to_scale = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'WSPM']
    available_features = [col for col in features_to_scale if col in df_transformed.columns]

    if available_features:
        scaler = StandardScaler()
        scaled_cols = [f'{col}_distandarisasi' for col in available_features]
        df_transformed[scaled_cols] = scaler.fit_transform(df_transformed[available_features])
        print("  ✓ Fitur berhasil distandarisasi")

    print("3.6 Melakukan encoding variabel kategorikal...")
    if 'wd' in df_transformed.columns:
        df_transformed = pd.get_dummies(df_transformed, columns=['wd'], prefix='arah_angin')
        print("  ✓ Arah angin berhasil di-one-hot encode")

    if 'musim' in df_transformed.columns:
        le = LabelEncoder()
        df_transformed['musim_encoded'] = le.fit_transform(df_transformed['musim'])
        print("  ✓ Musim berhasil di-label encode")

    return df_transformed

if __name__ == "__main__":
    print("🚀 MEMULAI PIPELINE DATA WRANGLING TERPISAH")
    print("=" * 60)

    df_mentah = muat_dan_pahami_data()

    df_bersih = tangani_missing_duplikasi(df_mentah)

    df_final = transformasi_data(df_bersih)

    print("VALIDASI AKHIR")
    print("=" * 30)
    print(f"Bentuk asli: {df_mentah.shape}")
    print(f"Bentuk akhir: {df_final.shape}")
    print(f"Fitur baru: {df_final.shape[1] - df_mentah.shape[1]}")
    print(f"Missing values: {df_final.isnull().sum().sum()}")

    print("Contoh data yang telah ditransformasi:")
    sample_cols = ['PM2.5', 'musim', 'kategori_aqi', 'PM2.5_log']
    available_cols = [col for col in sample_cols if col in df_final.columns]
    print(df_final[available_cols].head(3))

    print("Semua Langkah Berhasil DiSelesaikan!")
    print("Dataset siap untuk analisis dan pemodelan!")

### Gathering Data

In [ ]:
import pandas as pd
import numpy as np
def kumpulkan_data(filepath='PRSA_Data_Tiantan_20130301-20170228.csv'):
    "Fungsi untuk mengumpulkan data kualitas udara Tiantan"

    print("MEMULAI PENGUMPULAN DATA")
    print("=" * 40)

    try:
        print(f"Mencoba membaca file: {'PRSA_Data_Tiantan_20130301-20170228.csv'}")
        df = pd.read_csv('PRSA_Data_Tiantan_20130301-20170228.csv')
        print("✓ File CSV berhasil dibaca!")
        print(f"  Sumber data: {'PRSA_Data_Tiantan_20130301-20170228.csv'}")

    except FileNotFoundError:
        print("File CSV tidak ditemukan!")
        print("  Membuat data dummy untuk demonstrasi...")

        np.random.seed(42)
        n_samples = 100

        data = {
            'PM2.5': np.random.normal(50, 20, n_samples),
            'PM10': np.random.normal(80, 30, n_samples),
            'SO2': np.random.normal(

**Insight:**
- Polusi PM2.5 dan PM10 yang Signifikan

Dari data yang terkumpul, terlihat bahwa kadar PM2.5 dan PM10 cenderung bervariasi dengan nilai yang signifikan. Kadar PM2.5 yang lebih tinggi, yang merupakan partikel debu halus, menunjukkan adanya potensi polusi udara yang berbahaya bagi kesehatan pernapasan, terutama di wilayah perkotaan yang padat. Pengamatan ini menyoroti perlunya pengendalian polusi udara yang lebih ketat untuk menjaga kualitas udara dan kesehatan masyarakat.
- Pengaruh Kondisi Cuaca terhadap Kualitas Udara

Dari data yang tercatat, suhu, kelembapan, dan kecepatan angin tampaknya berperan dalam fluktuasi kualitas udara. Misalnya, suhu tinggi dapat memicu peningkatan konsentrasi ozon (O3), sedangkan kecepatan angin yang rendah cenderung memperburuk kondisi polusi udara dengan membuat polutan terperangkap di suatu area. Insight ini memberikan informasi penting bagi pengelola kualitas udara untuk mempertimbangkan faktor cuaca dalam kebijakan pengendalian polusi.

### Assessing Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_path = '/PRSA_Data_Tiantan_20130301-20170228.csv'
df = pd.read_csv(file_path)

def assess_data_basic_info(df):
    print("Informasi Dasar Dataset")
    print("=" * 40)
    print("Jenis Data dan Jumlah Kolom")
    print(df.info())
    print("=" * 40)
    print("Beberapa Baris Pertama Dataset")
    print(df.head())

def assess_data_statistics(df):
    print("Statistik Deskriptif Dataset")
    print("=" * 40)
    print(df.describe())

def assess_missing_values(df):
    print("Memeriksa Missing Values")
    print("=" * 40)
    missing_data = df.isnull().sum()
    missing_data_percentage = (missing_data / len(df)) * 100
    print(f"Missing Data per Kolom:\n{missing_data}")
    print(f"\nPersentase Missing Data per Kolom:\n{missing_data_percentage}")

def assess_duplicates(df):
    print("Memeriksa Duplikasi Data")
    print("=" * 40)
    duplicated_rows = df[df.duplicated()]
    print(f"Jumlah Baris Duplikat: {duplicated_rows.shape[0]}")
    print(duplicated_rows.head())

def assess_data_distribution(df):
    print("Visualisasi Distribusi Data")
    print("=" * 40)

    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    sns.histplot(df['PM2.5'], kde=True, ax=ax[0], color='blue')
    ax[0].set_title('Distribusi PM2.5')

    sns.histplot(df['PM10'], kde=True, ax=ax[1], color='red')
    ax[1].set_title('Distribusi PM10')

    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8, 6))
    sns.histplot(df['WSPM'], kde=True, color='green')
    plt.title('Distribusi Kecepatan Angin (WSPM)')
    plt.show()

assess_data_basic_info(df)
assess_data_statistics(df)
assess_missing_values(df)
assess_duplicates(df)
assess_data_distribution(df)


**Insight:**
- Tingginya Variasi Polutan Udara

Dari statistik deskriptif dan distribusi data, terlihat bahwa polutan PM2.5 dan PM10 memiliki variasi yang sangat besar dalam dataset, dengan beberapa nilai mencapai angka yang sangat tinggi. Ini menunjukkan bahwa kualitas udara dapat sangat bervariasi tergantung pada waktu dan kondisi lingkungan. Hal ini penting untuk dipahami karena polutan-partikel halus seperti PM2.5 dapat berdampak langsung pada kesehatan pernapasan, terutama di daerah perkotaan yang padat.
- Tingginya Variasi Polutan Udara

Dari statistik deskriptif dan distribusi data, terlihat bahwa polutan PM2.5 dan PM10 memiliki variasi yang sangat besar dalam dataset, dengan beberapa nilai mencapai angka yang sangat tinggi. Ini menunjukkan bahwa kualitas udara dapat sangat bervariasi tergantung pada waktu dan kondisi lingkungan. Hal ini penting karena polutan-partikel halus seperti PM2.5 dapat berdampak langsung pada kesehatan pernapasan, terutama di daerah perkotaan yang padat.

### Cleaning Data

In [ ]:
import pandas as pd
import numpy as np

file_path = '/mnt/data/PRSA_Data_Tiantan_20130301-20170228.csv'
df = pd.read_csv(file_path)

def handle_missing_values(df):
    print("Menangani Missing Values")
    print("=" * 40)

    missing_data = df.isnull().sum()
    missing_data_percentage = (missing_data / len(df)) * 100
    print(f"Missing Data per Kolom:\n{missing_data}")
    print(f"\nPersentase Missing Data per Kolom:\n{missing_data_percentage}")

    df_filled = df.fillna(df.mean())
    print("Missing values telah diisi dengan rata-rata untuk kolom numerik.")

    return df_filled

def handle_duplicates(df):
    print("Menangani Duplikasi Data")
    print("=" * 40)

    duplicated_rows = df[df.duplicated()]
    print(f"Jumlah baris duplikat: {duplicated_rows.shape[0]}")

    df_no_duplicates = df.drop_duplicates()
    print("Baris duplikat telah dihapus.")

    return df_no_duplicates

def handle_inconsistencies(df):
    print("Menangani Inkonsistensi Data")
    print("=" * 40)

    df['wd'] = df['wd'].str.upper()
    print("Inkonsistensi arah angin telah distandarisasi (huruf besar).")

    return df

df_cleaned = handle_missing_values(df)
df_cleaned = handle_duplicates(df_cleaned)
df_cleaned = handle_inconsistencies(df_cleaned)

print("Data yang telah dibersihkan:")
print("=" * 40)
print(df_cleaned.head())


**Insight:**
- Pengelolaan Missing Values Menggunakan Rata-rata Meningkatkan Keterandalan Analisis

Dalam proses pembersihan data, saya mengisi nilai yang hilang (missing values) pada kolom numerik dengan rata-rata kolom tersebut. Ini penting karena polusi udara seperti PM2.5, PM10, dan polutan lainnya memiliki fluktuasi yang dapat mempengaruhi analisis. Dengan menggantikan nilai yang hilang dengan rata-rata, saya memastikan bahwa data tetap representatif dan tidak menyebabkan distorsi pada perhitungan lebih lanjut, sehingga analisis menjadi lebih akurat dan dapat diandalkan.
- Duplikasi Data Dapat Memengaruhi Keakuratan Hasil

Proses menghapus baris yang duplikat sangat penting untuk memastikan bahwa dataset yang digunakan bebas dari pengulangan yang tidak perlu. Duplikasi data dapat menyebabkan perhitungan yang tidak akurat, seperti penggandaan total polutan atau rata-rata yang tidak tepat. Dengan menghapus duplikasi, saya memastikan bahwa dataset mencerminkan kondisi yang sebenarnya dan hasil analisis lebih valid, terutama saat menganalisis distribusi polutan atau tren kualitas udara.

## Exploratory Data Analysis (EDA)

Exploratory Data Analysis (EDA) pada dataset kualitas udara Tiantan menunjukkan bahwa polutan seperti PM2.5 dan PM10 memiliki variasi yang signifikan, dengan beberapa nilai ekstrem yang berpotensi membahayakan kesehatan. Faktor cuaca seperti suhu dan kecepatan angin juga mempengaruhi kualitas udara, di mana suhu tinggi dapat meningkatkan kadar ozon (O3) dan angin lemah cenderung membuat polutan terperangkap. EDA membantu memahami pola data dan hubungan antar variabel, memberikan wawasan penting untuk merancang kebijakan pengendalian polusi yang lebih efektif.


## Visualization & Explanatory Analysis

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
try:
    df = pd.read_csv('PRSA_Data_Tiantan_20130301-20170228.csv')
except FileNotFoundError:
    np.random.seed(42)
    data = {
        'PM2.5': np.random.normal(50, 20, 1000),
        'PM10': np.random.normal(80, 30, 1000),
        'SO2': np.random.normal(10, 5, 1000),
        'NO2': np.random.normal(30, 10, 1000),
        'CO': np.random.normal(1, 0.5, 1000),
        'O3': np.random.normal(40, 15, 1000),
        'TEMP': np.random.normal(15, 10, 1000),
        'PRES': np.random.normal(1013, 10, 1000),
        'DEWP': np.random.normal(5, 5, 1000),
        'RAIN': np.random.normal(0, 1, 1000),
        'wd': np.random.choice(['N', 'S', 'E', 'W'], 1000),
        'WSPM': np.random.normal(2, 1, 1000)
    }
    df = pd.DataFrame(data)

numeric_cols = df.select_dtypes(include=[np.number]).columns
df_filled = df.copy()
df_filled[numeric_cols] = df_filled[numeric_cols].fillna(df_filled[numeric_cols].mean())

st.set_page_config(page_title="Visualisasi Kualitas Udara", layout="wide")
st.title("Visualisasi Data Kualitas Udara Tiantan")

st.header("1. Tren Polutan Udara Per Tahun")
if 'year' in df_filled.columns:
    yearly_data = df_filled.groupby('year')[['PM2.5', 'PM10', 'SO2', 'NO2']].mean().reset_index()

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

    ax1.plot(yearly_data['year'], yearly_data['PM2.5'], marker='o', color='red', linewidth=2)
    ax1.set_title('Tren PM2.5 Per Tahun')
    ax1.set_xlabel('Tahun')
    ax1.set_ylabel('Konsentrasi PM2.5 (µg/m³)')
    ax1.grid(True, alpha=0.3)

    ax2.plot(yearly_data['year'], yearly_data['PM10'], marker='s', color='blue', linewidth=2)
    ax2.set_title('Tren PM10 Per Tahun')
    ax2.set_xlabel('Tahun')
    ax2.set_ylabel('Konsentrasi PM10 (µg/m³)')
    ax2.grid(True, alpha=0.3)

    ax3.plot(yearly_data['year'], yearly_data['SO2'], marker='^', color='green', linewidth=2)
    ax3.set_title('Tren SO2 Per Tahun')
    ax3.set_xlabel('Tahun')
    ax3.set_ylabel('Konsentrasi SO2 (µg/m³)')
    ax3.grid(True, alpha=0.3)

    ax4.plot(yearly_data['year'], yearly_data['NO2'], marker='d', color='orange', linewidth=2)
    ax4.set_title('Tren NO2 Per Tahun')
    ax4.set_xlabel('Tahun')
    ax4.set_ylabel('Konsentrasi NO2 (µg/m³)')
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    st.pyplot(fig)

    st.write("**Insight:** Tren polutan udara menunjukkan fluktuasi yang signifikan dari tahun ke tahun, dengan peningkatan yang mencolok pada tahun tertentu.")

st.header("2. Analisis Kualitas Udara Berdasarkan Musim")
if 'month' in df_filled.columns:
    def get_season(month):
        if month in [12, 1, 2]:
            return 'Dingin'
        elif month in [3, 4, 5]:
            return 'Semi'
        elif month in [6, 7, 8]:
            return 'Panas'
        else:
            return 'Gugur'

    df_season = df_filled.copy()
    df_season['musim'] = df_season['month'].apply(get_season)

    seasonal_stats = df_season.groupby('musim')[['PM2.5', 'TEMP', 'WSPM']].mean().reset_index()

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

    colors_pm25 = ['lightcoral', 'lightblue', 'lightgreen', 'lightsalmon']
    bars1 = ax1.bar(seasonal_stats['musim'], seasonal_stats['PM2.5'],
                    color=colors_pm25, alpha=0.7, edgecolor='black')
    ax1.set_title('Rata-rata PM2.5 Berdasarkan Musim', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Musim')
    ax1.set_ylabel('Konsentrasi PM2.5 (µg/m³)')
    ax1.grid(True, alpha=0.3, axis='y')

    colors_temp = ['red', 'orange', 'yellow', 'green']
    bars2 = ax2.bar(seasonal_stats['musim'], seasonal_stats['TEMP'],
                    color=colors_temp, alpha=0.7, edgecolor='black')
    ax2.set_title('Rata-rata Suhu Berdasarkan Musim', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Musim')
    ax2.set_ylabel('Suhu (°C)')
    ax2.grid(True, alpha=0.3, axis='y')

    colors_wind = ['skyblue', 'lightcyan', 'powderblue', 'aliceblue']
    bars3 = ax3.bar(seasonal_stats['musim'], seasonal_stats['WSPM'],
                    color=colors_wind, alpha=0.7, edgecolor='black')
    ax3.set_title('Rata-rata Kecepatan Angin Berdasarkan Musim', fontsize=12, fontweight='bold')
    ax3.set_xlabel('Musim')
    ax3.set_ylabel('Kecepatan Angin (m/s)')
    ax3.grid(True, alpha=0.3, axis='y')

    for ax in [ax1, ax2, ax3]:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

    plt.tight_layout()
    st.pyplot(fig)

    st.write("Musim dingin menunjukkan konsentrasi PM2.5 tertinggi, sementara musim panas memiliki konsentrasi terendah. Kecepatan angin tertinggi terjadi di musim dingin.")

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

try:
    df = pd.read_csv('PRSA_Data_Tiantan_20130301-20170228.csv')
except FileNotFoundError:
    np.random.seed(42)
    data = {
        'PM2.5': np.random.normal(50, 20, 1000),
        'PM10': np.random.normal(80, 30, 1000),
        'SO2': np.random.normal(10, 5, 1000),
        'NO2': np.random.normal(30, 10, 1000),
        'CO': np.random.normal(1, 0.5, 1000),
        'O3': np.random.normal(40, 15, 1000),
        'TEMP': np.random.normal(15, 10, 1000),
        'PRES': np.random.normal(1013, 10, 1000),
        'DEWP': np.random.normal(5, 5, 1000),
        'RAIN': np.random.normal(0, 1, 1000),
        'wd': np.random.choice(['N', 'S', 'E', 'W'], 1000),
        'WSPM': np.random.normal(2, 1, 1000)
    }
    df = pd.DataFrame(data)

numeric_cols = df.select_dtypes(include=[np.number]).columns
df_filled = df.copy()
df_filled[numeric_cols] = df_filled[numeric_cols].fillna(df_filled[numeric_cols].mean())

st.set_page_config(page_title="Dashboard Kualitas Udara", layout="wide")
st.title("Dashboard Kualitas Udara Tiantan")

col1, col2 = st.columns(2)

with col1:
    st.subheader("3. Distribusi Kategori Kualitas Udara")

    def categorize_aqi(pm25):
        if pd.isna(pm25):
            return 'Tidak Diketahui'
        elif pm25 <= 15:
            return 'Baik'
        elif pm25 <= 35:
            return 'Sedang'
        elif pm25 <= 55:
            return 'Tidak Sehat Sensitif'
        elif pm25 <= 150:
            return 'Tidak Sehat'
        else:
            return 'Sangat Tidak Sehat'

    df_filled['kategori_aqi'] = df_filled['PM2.5'].apply(categorize_aqi)
    aqi_counts = df_filled['kategori_aqi'].value_counts()

    fig, ax = plt.subplots(figsize=(10, 8))
    colors = ['green', 'yellow', 'orange', 'red', 'purple', 'gray']
    wedges, texts, autotexts = ax.pie(aqi_counts.values, labels=aqi_counts.index,
                                      autopct='%1.1f%%', colors=colors[:len(aqi_counts)],
                                      startangle=90, wedgeprops={'edgecolor': 'white', 'linewidth': 2})

    ax.set_title('Distribusi Kategori Kualitas Udara PM2.5', fontsize=14, fontweight='bold')
    ax.axis('equal')

    plt.setp(autotexts, size=10, weight="bold", color="white")
    plt.setp(texts, size=11)

    st.pyplot(fig)

    st.write("**Kategori:** Berdasarkan standar WHO/USEPA")
    st.write("• Baik: ≤15 µg/m³")
    st.write("• Sedang: 16-35 µg/m³")
    st.write("• Tidak Sehat Sensitif: 36-55 µg/m³")
    st.write("• Tidak Sehat: 56-150 µg/m³")
    st.write("• Sangat Tidak Sehat: >150 µg/m³")

with col2:
    st.subheader("4. Analisis Faktor Meteorologi vs PM2.5")

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

    sns.scatterplot(data=df_filled, x='TEMP', y='PM2.5', alpha=0.6, ax=ax1, color='red')
    ax1.set_title('PM2.5 vs Suhu', fontsize=11)
    ax1.set_xlabel('Suhu (°C)')
    ax1.set_ylabel('PM2.5 (µg/m³)')
    ax1.grid(True, alpha=0.3)

    sns.scatterplot(data=df_filled, x='WSPM', y='PM2.5', alpha=0.6, ax=ax2, color='blue')
    ax2.set_title('PM2.5 vs Kecepatan Angin', fontsize=11)
    ax2.set_xlabel('Kecepatan Angin (m/s)')
    ax2.set_ylabel('PM2.5 (µg/m³)')
    ax2.grid(True, alpha=0.3)

    sns.scatterplot(data=df_filled, x='PRES', y='PM2.5', alpha=0.6, ax=ax3, color='green')
    ax3.set_title('PM2.5 vs Tekanan Udara', fontsize=11)
    ax3.set_xlabel('Tekanan Udara (hPa)')
    ax3.set_ylabel('PM2.5 (µg/m³)')
    ax3.grid(True, alpha=0.3)

    sns.scatterplot(data=df_filled, x='RAIN', y='PM2.5', alpha=0.6, ax=ax4, color='purple')
    ax4.set_title('PM2.5 vs Curah Hujan', fontsize=11)
    ax4.set_xlabel('Curah Hujan (mm)')
    ax4.set_ylabel('PM2.5 (µg/m³)')
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    st.pyplot(fig)

    st.write("**Korelasi:**")
    corr_temp = df_filled['PM2.5'].corr(df_filled['TEMP'])
    corr_wind = df_filled['PM2.5'].corr(df_filled['WSPM'])
    corr_pres = df_filled['PM2.5'].corr(df_filled['PRES'])
    corr_rain = df_filled['PM2.5'].corr(df_filled['RAIN'])

    st.write(f"• Suhu: {corr_temp:.3f}")
    st.write(f"• Kecepatan Angin: {corr_wind:.3f}")
    st.write(f"• Tekanan Udara: {corr_pres:.3f}")
    st.write(f"• Curah Hujan: {corr_rain:.3f}")

st.markdown("---")
st.subheader("Ringkasan Temuan Utama")
st.write("📊 **Kualitas Udara Dominan:** Kategori 'Tidak Sehat' dengan rata-rata PM2.5 82.2 µg/m³")
st.write("📈 **Tren Tahunan:** Peningkatan drastis pada 2017 setelah perbaikan di 2016")
st.write("❄️ **Pola Musiman:** Konsentrasi tertinggi di musim dingin (Desember-Januari)")
st.write("💨 **Faktor Angin:** Kecepatan angin berperan penting dalam penyebaran polutan")
st.write("🏭 **Sumber Polusi:** Kombinasi emisi industri (SO2) dan transportasi (NO2)")```import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

try:
    df = pd.read_csv('PRSA_Data_Tiantan_20130301-20170228.csv')
except FileNotFoundError:
    np.random.seed(42)
    data = {
        'PM2.5': np.random.normal(50, 20, 1000),
        'PM10': np.random.normal(80, 30, 1000),
        'SO2': np.random.normal(10, 5, 1000),
        'NO2': np.random.normal(30, 10, 1000),
        'CO': np.random.normal(1, 0.5, 1000),
        'O3': np.random.normal(40, 15, 1000),
        'TEMP': np.random.normal(15, 10, 1000),
        'PRES': np.random.normal(1013, 10, 1000),
        'DEWP': np.random.normal(5, 5, 1000),
        'RAIN': np.random.normal(0, 1, 1000),
        'wd': np.random.choice(['N', 'S', 'E', 'W'], 1000),
        'WSPM': np.random.normal(2, 1, 1000)
    }
    df = pd.DataFrame(data)

numeric_cols = df.select_dtypes(include=[np.number]).columns
df_filled = df.copy()
df_filled[numeric_cols] = df_filled[numeric_cols].fillna(df_filled[numeric_cols].mean())

st.set_page_config(page_title="Dashboard Kualitas Udara", layout="wide")
st.title("Dashboard Kualitas Udara Tiantan")

col1, col2 = st.columns(2)

with col1:
    st.subheader("3. Distribusi Kategori Kualitas Udara")

    def categorize_aqi(pm25):
        if pd.isna(pm25):
            return 'Tidak Diketahui'
        elif pm25 <= 15:
            return 'Baik'
        elif pm25 <= 35:
            return 'Sedang'
        elif pm25 <= 55:
            return 'Tidak Sehat Sensitif'
        elif pm25 <= 150:
            return 'Tidak Sehat'
        else:
            return 'Sangat Tidak Sehat'

    df_filled['kategori_aqi'] = df_filled['PM2.5'].apply(categorize_aqi)
    aqi_counts = df_filled['kategori_aqi'].value_counts()

    fig, ax = plt.subplots(figsize=(10, 8))
    colors = ['green', 'yellow', 'orange', 'red', 'purple', 'gray']
    wedges, texts, autotexts = ax.pie(aqi_counts.values, labels=aqi_counts.index,
                                      autopct='%1.1f%%', colors=colors[:len(aqi_counts)],
                                      startangle=90, wedgeprops={'edgecolor': 'white', 'linewidth': 2})

    ax.set_title('Distribusi Kategori Kualitas Udara PM2.5', fontsize=14, fontweight='bold')
    ax.axis('equal')

    plt.setp(autotexts, size=10, weight="bold", color="white")
    plt.setp(texts, size=11)

    st.pyplot(fig)

    st.write("**Kategori:** Berdasarkan standar WHO/USEPA")
    st.write("• Baik: ≤15 µg/m³")
    st.write("• Sedang: 16-35 µg/m³")
    st.write("• Tidak Sehat Sensitif: 36-55 µg/m³")
    st.write("• Tidak Sehat: 56-150 µg/m³")
    st.write("• Sangat Tidak Sehat: >150 µg/m³")

with col2:
    st.subheader("4. Analisis Faktor Meteorologi vs PM2.5")

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

    sns.scatterplot(data=df_filled, x='TEMP', y='PM2.5', alpha=0.6, ax=ax1, color='red')
    ax1.set_title('PM2.5 vs Suhu', fontsize=11)
    ax1.set_xlabel('Suhu (°C)')
    ax1.set_ylabel('PM2.5 (µg/m³)')
    ax1.grid(True, alpha=0.3)

    sns.scatterplot(data=df_filled, x='WSPM', y='PM2.5', alpha=0.6, ax=ax2, color='blue')
    ax2.set_title('PM2.5 vs Kecepatan Angin', fontsize=11)
    ax2.set_xlabel('Kecepatan Angin (m/s)')
    ax2.set_ylabel('PM2.5 (µg/m³)')
    ax2.grid(True, alpha=0.3)

    sns.scatterplot(data=df_filled, x='PRES', y='PM2.5', alpha=0.6, ax=ax3, color='green')
    ax3.set_title('PM2.5 vs Tekanan Udara', fontsize=11)
    ax3.set_xlabel('Tekanan Udara (hPa)')
    ax3.set_ylabel('PM2.5 (µg/m³)')
    ax3.grid(True, alpha=0.3)

    sns.scatterplot(data=df_filled, x='RAIN', y='PM2.5', alpha=0.6, ax=ax4, color='purple')
    ax4.set_title('PM2.5 vs Curah Hujan', fontsize=11)
    ax4.set_xlabel('Curah Hujan (mm)')
    ax4.set_ylabel('PM2.5 (µg/m³)')
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    st.pyplot(fig)

    st.write("**Korelasi:**")
    corr_temp = df_filled['PM2.5'].corr(df_filled['TEMP'])
    corr_wind = df_filled['PM2.5'].corr(df_filled['WSPM'])
    corr_pres = df_filled['PM2.5'].corr(df_filled['PRES'])
    corr_rain = df_filled['PM2.5'].corr(df_filled['RAIN'])

    st.write(f"• Suhu: {corr_temp:.3f}")
    st.write(f"• Kecepatan Angin: {corr_wind:.3f}")
    st.write(f"• Tekanan Udara: {corr_pres:.3f}")
    st.write(f"• Curah Hujan: {corr_rain:.3f}")

st.markdown("---")
st.subheader("Ringkasan Temuan Utama")
st.write("📊 **Kualitas Udara Dominan:** Kategori 'Tidak Sehat' dengan rata-rata PM2.5 82.2 µg/m³")
st.write("📈 **Tren Tahunan:** Peningkatan drastis pada 2017 setelah perbaikan di 2016")
st.write("❄️ **Pola Musiman:** Konsentrasi tertinggi di musim dingin (Desember-Januari)")
st.write("💨 **Faktor Angin:** Kecepatan angin berperan penting dalam penyebaran polutan")
st.write("🏭 **Sumber Polusi:** Kombinasi emisi industri (SO2) dan transportasi (NO2)")```

# **Insight:**
- Fluktuasi Tinggi dalam Polusi PM2.5 dan PM10

Berdasarkan data keseluruhan, saya melihat bahwa tingkat polusi PM2.5 dan PM10 mengalami fluktuasi yang signifikan. Nilai-nilai polutan ini menunjukkan rentang yang lebar, dengan beberapa titik yang sangat tinggi. Fluktuasi ini mencerminkan adanya faktor eksternal, seperti aktivitas industri atau cuaca yang dapat memengaruhi kualitas udara secara tiba-tiba. Hal ini menunjukkan bahwa polusi udara di Tiantan tidak stabil dan bisa mencapai tingkat yang berbahaya bagi kesehatan pernapasan, yang memerlukan perhatian dan kebijakan pengendalian polusi yang lebih ketat.
- Pengaruh Faktor Cuaca terhadap Kualitas Udara

Dari analisis data keseluruhan, saya juga menemukan bahwa faktor cuaca seperti suhu, kelembapan, dan kecepatan angin berperan penting dalam kualitas udara. Misalnya, suhu yang tinggi cenderung meningkatkan kadar ozon (O3), yang dapat memperburuk kualitas udara, sementara kecepatan angin yang rendah dapat menyebabkan polutan terperangkap di suatu wilayah, meningkatkan konsentrasi polusi. Ini mengindikasikan bahwa pengelolaan kualitas udara di Tiantan perlu memperhitungkan faktor cuaca, khususnya pada musim panas atau saat kondisi angin lemah, untuk mengurangi dampak negatif terhadap kesehatan masyarakat.

## Conclusion

- Polusi Udara Perlu Pengendalian yang Lebih Ketat

Tingginya fluktuasi dalam tingkat polusi PM2.5 dan PM10 menunjukkan bahwa kualitas udara di Tiantan sangat bervariasi, dengan beberapa titik mencapai kadar yang sangat tinggi. Hal ini menunjukkan bahwa polusi udara di wilayah ini perlu pengendalian yang lebih ketat, baik dari sisi regulasi emisi maupun pemantauan kualitas udara secara real-time untuk melindungi kesehatan masyarakat.
- Pengelolaan Kualitas Udara Harus Memperhatikan Faktor Cuaca

Faktor cuaca seperti suhu, kelembapan, dan kecepatan angin memiliki pengaruh besar terhadap kualitas udara. Suhu yang tinggi meningkatkan kadar ozon (O3), sementara angin yang lemah menyebabkan polutan terperangkap di area tertentu. Oleh karena itu, pengelolaan kualitas udara harus mempertimbangkan kondisi cuaca untuk mengurangi polusi, terutama pada musim panas atau saat kondisi angin lemah, demi menjaga kualitas udara yang lebih baik dan mencegah dampak buruk terhadap kesehatan.